In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession


In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("HIGGS")
conf.set("spark.executor.memory", "20G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")
# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .appName("spark session example") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/24 22:01:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("delimiter", ",").option("header", "false").csv('/work/li.baol/data/HIGGS.csv')
df.show()

22/11/24 22:02:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|                 _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c

In [4]:
df.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28']

In [5]:
# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, '/work/li.baol/data/HIGGS.txt')
# Split the data into training and test sets (30% held out for testing)
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

22/11/24 21:52:45 WARN BlockManager: Putting block rdd_10_22 failed due to exception org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/pyspark/mllib/util.py", line 144, in <lambda>
    parsed = lines.map(lambda l: MLUtils._parse_libsvm_line(l))
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/mllib/util.py

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 22 in stage 2.0 failed 1 times, most recent failure: Lost task 22.0 in stage 2.0 (TID 72) (c0172 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2

In [4]:
# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                    impurity='variance', maxDepth=5, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(testData.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression tree model:')
print(model.toDebugString())

# Save and load model
# model.save(sc, "target/tmp/myDecisionTreeRegressionModel")
# sameModel = DecisionTreeModel.load(sc, "target/tmp/myDecisionTreeRegressionModel")

Test Mean Squared Error = 0.1
Learned regression tree model:
DecisionTreeModel regressor of depth 2 with 5 nodes
  If (feature 405 <= 21.0)
   If (feature 100 <= 193.5)
    Predict: 0.0
   Else (feature 100 > 193.5)
    Predict: 1.0
  Else (feature 405 > 21.0)
   Predict: 1.0



In [5]:
data

PythonRDD[31] at RDD at PythonRDD.scala:53